# Notebook 6: Systems Biology Analysis and Course Unit Mapping

## Overview
This final notebook synthesizes all our findings and explicitly maps them to the **5 units of the Systems Biology course**.

### Course Syllabus Coverage:
1. **Unit 1**: Network Organization (Motifs)
2. **Unit 2**: Design Principles  
3. **Unit 3**: Dynamic Modeling
4. **Unit 4**: Switches & Clocks
5. **Unit 5**: Robustness & Noise

### Project Summary:
We implemented a **memristive FitzHugh-Nagumo neuron model** to study **synaptic plasticity** and **Hebbian learning**. Our system demonstrates:
- Self-organized learning
- Adaptation through plasticity
- Robustness to perturbations
- Complex dynamics from simple rules

In [1]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import seaborn as sns

from config import FHN_PARAMS, LEARNING_PARAMS, VIZ_PARAMS
from utils import coupled_fhn_static, synchronization_index

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✓ Libraries loaded for final analysis")

✓ Libraries loaded for final analysis


---
## Unit 1: Network Organization (Motifs)

### Requirement:
> Model a network motif and analyze emergent behavior

### Our Implementation:

#### **2-Node Feed-Forward Motif (Teacher → Student)**

```
    Teacher (receives external input)
        |
        | M(t) - dynamic synaptic weight
        ↓
    Student (receives coupling)
```

#### Key Innovation: **Dynamic Edge**
- Traditional motifs: **Static edges** (fixed connection strength)
- Our model: **Adaptive edge** (weight evolves: $M(0)=0 \rightarrow M(\infty)\approx 1$)

#### Emergent Behavior:
1. **Associative Learning**: Student learns to follow teacher
2. **Synchronization**: Emerges without external control
3. **Self-Organization**: No supervisor needed

#### Complexity from Simplicity:
- Just **2 nodes**, but exhibits rich learning dynamics
- Demonstrates that even minimal networks can show **intelligent** behavior when edges are adaptive

### Biological Example:
**Pre-synaptic → Post-synaptic** connections in hippocampus (memory formation)

---

## Unit 2: Design Principles

### Requirement:
> Demonstrate a biological design principle (e.g., adaptation, robustness, modularity)

### Our Implementation: **ADAPTATION**

#### Definition:
> A system **adapts** when it modifies its internal structure to optimize performance in a changing environment.

#### How Our System Adapts:

**Initial State**: Neurons unconnected ($M=0$)
- Teacher and student oscillate independently
- No information transfer
- Inefficient

**Adaptation Process**: Hebbian plasticity
- System detects correlated activity
- Synaptic weight increases: $\frac{dM}{dt} = \alpha(\Delta v)^2(1-M) - \beta M$
- Connection strengthens automatically

**Final State**: Neurons synchronized ($M\approx 1$)
- Student tracks teacher
- Efficient information transfer
- **Optimized communication**

#### Energy Efficiency:
- **Before adaptation**: Student wastes energy on uncorrelated activity
- **After adaptation**: Student's activity is aligned with teacher → **energy-efficient**

#### Mathematical Evidence:
From Notebook 4:
- Synchronization index: $0.2 \rightarrow 0.95$
- Synaptic weight: $0 \rightarrow 1$
- Time to adapt: ~150 time units

### Connection to Biology:
**Spike-Timing-Dependent Plasticity (STDP)** in neural circuits:
- Neurons adjust connections based on temporal correlations
- Optimizes neural coding and information transmission
- Underlies learning and memory

---

## Unit 3: Dynamic Modeling

### Requirement:
> Solve a system of coupled differential equations and analyze phase space

### Our Implementation:

#### **Model 1: Continuous-Time (5D ODE System)**

**State variables**: $[v_1, w_1, v_2, w_2, M]$

**Equations**:
$$
\begin{aligned}
\frac{dv_1}{dt} &= v_1 - \frac{v_1^3}{3} - w_1 + I_{ext} \\
\frac{dw_1}{dt} &= \frac{v_1 + a - bw_1}{\tau} \\
\frac{dv_2}{dt} &= v_2 - \frac{v_2^3}{3} - w_2 + M(v_1 - v_2) \\
\frac{dw_2}{dt} &= \frac{v_2 + a - bw_2}{\tau} \\
\frac{dM}{dt} &= \alpha(v_1 - v_2)^2(1-M) - \beta M
\end{aligned}
$$

#### **Model 2: Discrete-Time (3D Map, Shatnawi et al. 2023)**

**Memristive FHN Map**:
$$
\begin{aligned}
x_{n+1} &= x_n - \frac{x_n^3}{3} - y_n + I_{ext} + k_1 z_n x_n \\
y_{n+1} &= \gamma y_n + \theta x_n + \delta \\
z_{n+1} &= z_n + \sin(z_n) - k_2 x_n
\end{aligned}
$$

#### Phase Space Analysis:

**Notebook 1**:
- 2D phase plane $(v, w)$
- Nullclines, fixed points, limit cycles
- Demonstrated excitable → oscillatory transition

**Notebook 3**:
- 3D attractor $(x, y, z)$
- Strange attractor geometry
- 2D projections reveal complex structure

#### Numerical Methods:
- **Continuous**: `scipy.integrate.odeint` (adaptive Runge-Kutta)
- **Discrete**: Iterative map evaluation
- Time step: $dt = 0.01$ for stability

#### Bifurcation Analysis:
- Varied coupling strength $g$: synchronization threshold
- Varied learning rate $\alpha$: speed of adaptation
- Varied external current $I_{ext}$: excitable ↔ oscillatory

---

## Unit 4: Switches & Clocks

### Requirement:
> Model a biological switch or oscillator

### Our Implementation: **TWO Types of Switches**

---

### Switch 1: **Excitable Neuron (FHN Model)**

#### Type: **Biological Switch** (Excitable Medium)

**Mechanism**:
- **Resting state**: Near fixed point $(v, w) \approx (-1.2, -0.6)$
- **Threshold**: Superthreshold stimulus → action potential
- **Refractory period**: Cannot fire again immediately
- **Reset**: Returns to rest

**Demonstration** (Notebook 1, Section 5):
- Brief pulse → single spike → return to rest
- Classic **all-or-none** response
- Threshold-based activation

#### Biological Relevance:
**Neuronal action potentials**: Na⁺ and K⁺ channel dynamics in axons

---

### Switch 2: **Memristor (Conductance Switch)**

#### Type: **Bistable Memory Switch**

**Mechanism**:
The memristor state $z$ acts as a **conductance switch** with memory:
- **State equation**: $z_{n+1} = z_n + \sin(z_n) - k_2 x_n$
- **Conductance**: $G(z) = z_n$ (memristor state)
- **Current**: $i = G(z) \cdot v$ (state-dependent)

**Power-Off Plot (POP)** (Notebook 3):
- Multiple stable equilibria at $z = k\pi$
- **Nonvolatile memory**: State persists without power
- Used in computing and neuromorphic chips

**Multistability**:
- Different initial $z_0$ → different hysteresis loops
- Each stable state is a distinct "memory"

#### Synaptic Interpretation:
The memristor switches the synapse between:
- **OFF state** ($M \approx 0$): No connection
- **ON state** ($M \approx 1$): Strong connection

This is analogous to **Long-Term Potentiation/Depression (LTP/LTD)** in synapses.

---

### Clock: **FHN Oscillator (Limit Cycle)**

#### Type: **Biological Clock**

**Mechanism**:
- For $I_{ext} > I_{threshold}$, neuron enters **limit cycle**
- Periodic firing: **biological clock**
- Period depends on $\tau$ (time constant)

**Demonstration** (Notebook 1, Section 4):
- $I_{ext} = 0.5$ → sustained oscillations
- Frequency ≈ 0.15 Hz (from simulations)

#### Biological Examples:
- **Cardiac pacemaker cells**: Rhythmic heartbeat
- **Central pattern generators**: Walking, breathing
- **Circadian clocks**: Sleep-wake cycles

---

## Unit 5: Robustness & Noise

### Requirement:
> Test system robustness to parameter variations and noise

### Our Implementation (Notebook 5):

---

### Test 1: **Parameter Mismatch**

**Question**: Can neurons with different parameters still synchronize?

**Method**:
- Teacher: standard parameters
- Student: $a, b, \tau$ varied by ±30%

**Results**:
- ✅ Learning succeeds up to 30% mismatch
- ✅ Final $M \geq 0.8$ (strong connection)
- ✅ Synchronization index $\geq 0.75$

**Conclusion**: System is **parameter-robust**

---

### Test 2: **Additive Noise**

**Question**: Can learning occur in a noisy environment?

**Method**:
- Add Gaussian noise to membrane potentials
- Noise strength: $\sigma = 0$ to $0.2$

**Results**:
- ✅ Moderate noise ($\sigma \leq 0.1$): Learning unaffected
- ✅ High noise ($\sigma = 0.2$): Learning still occurs, slower convergence

**Conclusion**: System is **noise-robust**

---

### Test 3: **Parameter Space Exploration**

**Question**: How sensitive is learning to $\alpha$ and $\beta$?

**Method**:
- Grid search: $\alpha \in [0.02, 0.3]$, $\beta \in [0.001, 0.05]$
- Measure final $M$ for each $(\alpha, \beta)$ pair

**Results**:
- ✅ **Wide parameter range** supports learning (green region in heatmap)
- ✅ System not fine-tuned (no narrow "knife-edge")
- ✅ Higher $\alpha/\beta$ ratio → better learning

**Conclusion**: System has **broad tolerance** to parameter tuning

---

### Test 4: **Recovery from Perturbations**

**Question**: Can a learned connection recover if damaged?

**Method**:
- Learn normally: $M: 0 \rightarrow 1$
- At $t=300$: Reset $M$ to $0.3$ (damage)
- Continue simulation

**Results**:
- ✅ System **re-learns** connection
- ✅ $M$ recovers to $\approx 1$ within 100 time units
- ✅ Demonstrates **homeostatic plasticity**

**Conclusion**: System exhibits **structural robustness**

---

### Summary: Three Types of Robustness

| Type | Definition | Our Evidence |
|------|------------|-------------|
| **Parameter Robustness** | Function maintained despite component variation | 30% parameter mismatch tolerated |
| **Environmental Robustness** | Function maintained in noisy conditions | Learning succeeds with σ=0.1 noise |
| **Structural Robustness** | Function recovers after damage | Synapse re-learned after reset |

### Why is the System Robust?

**Feedback mechanism**:
- Hebbian rule is **local** and **activity-dependent**
- Continuously adjusts based on ongoing dynamics
- No global controller → distributed robustness
- Self-correcting: errors are automatically compensated

This is a **hallmark of robust biological design**!

---

## Final Summary: Project Achievements

### What We Built:
1. ✅ **Single FHN Neuron**: Excitability, phase plane analysis
2. ✅ **Static Coupling**: Synchronization threshold
3. ✅ **Discrete Memristor**: Hysteresis loops, multistability
4. ✅ **Hebbian Learning**: Self-organized weight growth
5. ✅ **Robustness Testing**: Parameter, noise, and structural robustness
6. ✅ **Course Mapping**: All 5 units comprehensively covered

### Key Contributions:

#### Scientific:
- Reproduced findings from **Shatnawi et al. (2023)**
- Extended to continuous-time Hebbian plasticity
- Demonstrated learning without supervision

#### Educational:
- Concrete implementation of abstract concepts
- Bridges theory (ODEs) and biology (neurons)
- Reproducible, well-documented code

### Biological Insights:

**Central Finding**:
> **Simple plasticity rules + local feedback → complex adaptive behavior**

This demonstrates that:
- **Intelligence** can emerge from simple components
- **Learning** doesn't require a teacher
- **Robustness** arises from distributed control

### Broader Impact:

This model has applications in:
- **Neuroscience**: Understanding synaptic plasticity
- **AI/ML**: Bio-inspired learning algorithms
- **Neuromorphic Computing**: Hardware implementation of adaptive synapses
- **Robotics**: Self-organizing control systems

---

## Course Unit Checklist

### ✅ Unit 1: Network Organization (Motifs)
- **Motif**: 2-node feed-forward (Teacher → Student)
- **Innovation**: Dynamic edge (adaptive weight)
- **Emergent behavior**: Associative learning, synchronization
- **Notebook**: 2, 4

### ✅ Unit 2: Design Principles
- **Principle**: Adaptation
- **Mechanism**: Hebbian plasticity
- **Evidence**: $M: 0 \rightarrow 1$, energy-efficient communication
- **Notebook**: 4

### ✅ Unit 3: Dynamic Modeling
- **Models**: 5D ODE system, 3D discrete map
- **Analysis**: Phase space, nullclines, bifurcations
- **Methods**: Numerical integration, iterative maps
- **Notebook**: 1, 2, 3, 4

### ✅ Unit 4: Switches & Clocks
- **Switch 1**: Excitable neuron (threshold activation)
- **Switch 2**: Memristor (conductance switch, bistability)
- **Clock**: FHN oscillator (limit cycle)
- **Notebook**: 1, 3

### ✅ Unit 5: Robustness & Noise
- **Test 1**: Parameter mismatch (30% tolerance)
- **Test 2**: Additive noise (σ=0.1 robust)
- **Test 3**: Parameter space (wide tolerance)
- **Test 4**: Perturbation recovery (homeostasis)
- **Notebook**: 5

---

## ✨ **ALL COURSE UNITS COMPREHENSIVELY COVERED** ✨

---

## References and Further Reading

### Primary Reference:
1. **Shatnawi, M.T., et al. (2023)**. "A Multistable Discrete Memristor and Its Application to Discrete-Time FitzHugh–Nagumo Model." *Electronics*, 12(13), 2929.

### Foundational Papers:
2. **FitzHugh, R. (1961)**. "Impulses and Physiological States in Theoretical Models of Nerve Membrane." *Biophysical Journal*, 1(6), 445-466.

3. **Nagumo, J., et al. (1962)**. "An Active Pulse Transmission Line Simulating Nerve Axon." *Proceedings of the IRE*, 50(10), 2061-2070.

4. **Hebb, D.O. (1949)**. *The Organization of Behavior*. Wiley, New York.

5. **Chua, L.O. (1971)**. "Memristor—The Missing Circuit Element." *IEEE Transactions on Circuit Theory*, 18(5), 507-519.

### Textbooks:
6. **Alon, U. (2019)**. *An Introduction to Systems Biology: Design Principles of Biological Circuits* (2nd ed.). Chapman & Hall/CRC.

7. **Izhikevich, E.M. (2007)**. *Dynamical Systems in Neuroscience*. MIT Press.

8. **Strogatz, S.H. (2015)**. *Nonlinear Dynamics and Chaos* (2nd ed.). Westview Press.

---